# CV Lab project - Method#1: Frame Differencing

* datasets:
    * train set, in each folder contains f-c.png and the json of the homography...
    * validation set, contains f-c.png, homographies and true label
    * test set, for submission

* 7 frames for each of the 10 cameras à 70 images for each sample 
    * f $\in$ {0,1,2,3,4,5,6}, where 3 is the center frame and
    * c $\in$ B01,...,B05,G01,...,G05 ,where B01 is the center camera

# ref:
* https://debuggercafe.com/moving-object-detection-using-frame-differencing-with-opencv/

In [ ]:
import pandas as pd
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import re
from skimage import exposure
import imageio

## Load The Images


## Algorithms

![Frame Difference Object Detection](./steps_block_diagram.png)

In [ ]:
imgMasks = [ 'merg_avg_masked.png', 'merg_v1_masked.png',  'merg_v2_masked.png' ] #available list of masks
dirs = ['train','validation']
fns = ['train','valid']

choseMask = imgMasks[0]

colorSpaces = { 'name': ['BRG', 'LAB', 'YCrCb', 'HSV', 'LUV'] 

}


In [ ]:
#idea, use training sets to calculate the median background


def get_background (imgNameMask='', time_frames = 7):
    train_folder_pattern = re.compile('train-(.*)-(.*)')
    background = []
    for filename in os.listdir('./data/train'):
        if train_folder_pattern.match(filename) is not None:
            for tf in range(time_frames):
                fn = os.path.join('./data/train/'+filename,str(tf)+'-'+imgNameMask)
                #print(' ',fn,end='') #verbose
                background += [cv2.imread(fn)/255.0]

    # calculate the median
    #m = np.median(background, axis=0)

    m = np.mean(background, axis=0)

    return m




global_mean_bg = get_background(choseMask)

In [ ]:
#mask 2
mask = cv2.imread('data/mask.png',0)

background = cv2.bitwise_and(global_mean_bg,global_mean_bg,mask=mask)
print(background.shape)
#background = np.swapaxes(background, 0, 2)
#print(background.shape)

#plt.imshow(global_mean_bg)



In [ ]:
plt.figure(figsize=(20,12))
global_grey_background = cv2.cvtColor(np.float32(background), cv2.COLOR_BGR2GRAY)
plt.imshow(global_grey_background)

In [ ]:
plt.figure(figsize=(20,12))
plt.imshow(global_mean_bg)

In [ ]:
sigma = 10
plt.figure(figsize=(20,12))
test = gaussian(global_mean_bg,sigma, multichannel=True)
print(test.shape)
plt.imshow(test)


## Detection

Looping Over the Frames and Detecting Moving Objects

In [ ]:
def get_background_per_train_folder (imgMask, trainFolder='1-0', time_frames = 7):
    img = []
    for fn in os.listdir('./data/train/train-'+trainFolder):
        #for tf in range(time_frames):
        #    background += [cv2.imread(os.path.join('./data/train/'+filename,str(tf)+'-'+imgNameMask))/255.0]
        #if ('merg' not in fn) and ('homo' not in fn) and ('.png' in fn):
        if (imgMask in fn):
            print(fn)
            img += [cv2.imread('./data/train/train-'+trainFolder+'/'+fn)/255.0]
            


    # calculate the median
    #m = np.median(bg, axis=0)

    #get the mean of time frames
    m = np.mean(img, axis=0)

    return m, img


#select integration method

local_mean_bg, img = get_background_per_train_folder(choseMask)

mask = cv2.imread('data/mask.png',0)
plt.figure(figsize=(20,12))
background = cv2.bitwise_and(local_mean_bg,local_mean_bg,mask=mask)
print(background.shape)
#background = np.swapaxes(background, 0, 2)
#print(background.shape)
local_grey_background = cv2.cvtColor(np.float32(background), cv2.COLOR_BGR2GRAY) #PIL is RGB, cv2 is BGR
#
#plt.imshow(background)
plt.imshow(local_grey_background)

In [ ]:
#RGB Space
test_img = copy.deepcopy(img[0])
testlocalbg = copy.deepcopy(local_mean_bg)
testglobalbg = copy.deepcopy(global_mean_bg)

plt.figure(figsize=(42,42))
p_background = test_img #cv2.cvtColor(np.float32(background), cv2.COLOR_BGR2LAB) #PIL is RGB, cv2 is BGR

plt.subplot(3,4,1)
plt.imshow(p_background) #R
plt.subplot(3,4,2)
plt.imshow(p_background[:,:,0]) #B
plt.subplot(3,4,3)
plt.imshow(p_background[:,:,1]) #G
plt.subplot(3,4,4)
plt.imshow(p_background[:,:,2]) #R

#p_background = global_mean_bg #cv2.cvtColor(np.float32(background), cv2.COLOR_BGR2LAB) #PIL is RGB, cv2 is BGR
p_background = cv2.absdiff( np.uint8(test_img*255), np.uint8(testlocalbg*255) )
plt.subplot(3,4,5)
plt.imshow(p_background) #R
plt.subplot(3,4,6)
plt.imshow(p_background[:,:,0]) #B
plt.subplot(3,4,7)
plt.imshow(p_background[:,:,1]) #G
plt.subplot(3,4,8)
plt.imshow(p_background[:,:,2]) #R

p_background = cv2.absdiff( np.uint8(test_img*255), np.uint8(testglobalbg*255) ) #cv2.cvtColor(np.float32(background), cv2.COLOR_BGR2LAB) #PIL is RGB, cv2 is BGR
plt.subplot(3,4,9)
plt.imshow(p_background) #R
plt.subplot(3,4,10)
plt.imshow(p_background[:,:,0]) #B
plt.subplot(3,4,11)
plt.imshow(p_background[:,:,1]) #G
plt.subplot(3,4,12)
plt.imshow(p_background[:,:,2]) #R

In [ ]:
#get from validation
from skimage.filters import unsharp_mask, gaussian

mask = cv2.imread('data/mask.png',0)

def get_background_per_train_folder (imgMask, trainFolder='1-0', time_frames = 7):
    img = []
    for fn in os.listdir('./data/validation/valid-'+trainFolder):
        #for tf in range(time_frames):
        #    background += [cv2.imread(os.path.join('./data/train/'+filename,str(tf)+'-'+imgNameMask))/255.0]
        #if ('merg' not in fn) and ('homo' not in fn) and ('.png' in fn):
        if (imgMask in fn):
            print(fn)
            img += [cv2.imread('./data/validation/valid-'+trainFolder+'/'+fn)/255.0]
            


    # calculate the median
    #m = np.median(bg, axis=0)

    #get the mean of time frames
    m = np.mean(img, axis=0)

    return m, img


#select integration method

_, img = get_background_per_train_folder(choseMask)



#save the R from BGR space
test_imgs = copy.deepcopy(img)
testglobalbg = copy.deepcopy(global_mean_bg)
sigma=10
testglobalbg = gaussian(testglobalbg,sigma, multichannel=True, preserve_range=True)
testglobalbg_ = cv2.bitwise_and(testglobalbg,testglobalbg,mask=mask)

folder = './valid-1-0-gifs-Rspace/'
file = 'valid-1-0.gif'
with imageio.get_writer(folder+file, mode="I") as writer:
    for i,im_ori in enumerate(test_imgs):
        im = im_ori.copy()
        im = cv2.bitwise_and(im,im,mask=mask)
        im = cv2.absdiff( np.uint8(im*255), np.uint8(testglobalbg_*255) ) 

        im_ori = cv2.bitwise_and(im_ori,im_ori,mask=mask)
        

        #im = unsharp_mask(im, radius=100, amount=10)
        #im = gaussian(im, sigma=10, multichannel=True)
        #im = cv2.dilate(im, None, iterations=2)
        #if im.dtype == 'float':
        #    im = np.uint8(im*255)

        
        im = im[:,:,2]
        #print(im.mean(), im.max())

        threshold = np.uint8(im.max() * 0.5)
        _, im = cv2.threshold( im , threshold, 255, cv2.THRESH_BINARY)
        im = cv2.dilate(im, None, iterations=3)
        
        contours, hierarchy = cv2.findContours( im, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        for contour in contours:
            # continue through the loop if contour area is less than 500...
            # ... helps in removing noise detection
            if cv2.contourArea(contour) < 120:
                continue
            # get the xmin, ymin, width, and height coordinates from the contours
            (x, y, w, h) = cv2.boundingRect(contour)
            print(x,y,w,h)
            # draw the bounding boxes
            cv2.rectangle(im_ori, (x, y), (x+w, y+h), (1.0, 0, 0), 2)

            #true Label
            x, y, w, h = 645, 473, 48, 37 
            cv2.rectangle(im_ori, (x, y), (x+w, y+h), (0.0, 1.0, 0), 2)
        
        

        #plt.imsave(folder+str(i+1)+'.png', im[:,:,2]) #ori

        im = im_ori #cv2.cvtColor(im_ori,cv2.COLOR_BGR2RGB)
        plt.imsave(folder+str(i+1)+'.png', im)

        tosave = plt.imread(folder+str(i+1)+'.png')
        #print("Adding frame to GIF file: ", idx + 1)
        writer.append_data(tosave)


In [ ]:
#RGB Space
test_img = copy.deepcopy(img[0])
testlocalbg = copy.deepcopy(local_mean_bg)
testglobalbg = copy.deepcopy(global_mean_bg)

plt.figure(figsize=(42,42))
p_background = test_img #cv2.cvtColor(np.float32(background), cv2.COLOR_BGR2LAB) #PIL is RGB, cv2 is BGR

plt.subplot(3,4,1)
plt.imshow(p_background) #R
plt.subplot(3,4,2)
plt.imshow(p_background[:,:,0]) #B
plt.subplot(3,4,3)
plt.imshow(p_background[:,:,1]) #G
plt.subplot(3,4,4)
plt.imshow(p_background[:,:,2]) #R

#p_background = global_mean_bg #cv2.cvtColor(np.float32(background), cv2.COLOR_BGR2LAB) #PIL is RGB, cv2 is BGR
p_background = cv2.absdiff( np.uint8(test_img*255), np.uint8(testlocalbg*255) )
plt.subplot(3,4,5)
plt.imshow(p_background) #R
plt.subplot(3,4,6)
plt.imshow(p_background[:,:,0]) #B
plt.subplot(3,4,7)
plt.imshow(p_background[:,:,1]) #G
plt.subplot(3,4,8)
plt.imshow(p_background[:,:,2]) #R

p_background = cv2.absdiff( np.uint8(test_img*255), np.uint8(testglobalbg*255) ) #cv2.cvtColor(np.float32(background), cv2.COLOR_BGR2LAB) #PIL is RGB, cv2 is BGR
plt.subplot(3,4,9)
plt.imshow(p_background) #R
plt.subplot(3,4,10)
plt.imshow(p_background[:,:,0]) #B
plt.subplot(3,4,11)
plt.imshow(p_background[:,:,1]) #G
plt.subplot(3,4,12)
plt.imshow(p_background[:,:,2]) #R

In [ ]:
#save the R from BGR space
test_imgs = copy.deepcopy(img)
testglobalbg = copy.deepcopy(global_mean_bg)
fn = './train-1-0-gifs-Rspace/train-1-0.gif'
with imageio.get_writer(fn, mode="I") as writer:
    for i,im_ori in enumerate(test_imgs):
        im = im_ori.copy()
        im = cv2.absdiff( np.uint8(im*255), np.uint8(testglobalbg*255) ) 

        im = im[:,:,2]
        _, im = cv2.threshold( im , 50, 255, cv2.THRESH_BINARY)
        im = cv2.dilate(im, None, iterations=2)
        
        contours, hierarchy = cv2.findContours( im, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        for contour in contours:
            # continue through the loop if contour area is less than 500...
            # ... helps in removing noise detection
            if cv2.contourArea(contour) < 70:
                continue
            # get the xmin, ymin, width, and height coordinates from the contours
            (x, y, w, h) = cv2.boundingRect(contour)
            print(x,y,w,h)
            # draw the bounding boxes
            cv2.rectangle(im_ori, (x, y), (x+w, y+h), (1.0, 0, 0), 2)

        #plt.imsave('./train-1-0-gifs-Rspace/'+str(i+1)+'.png', im[:,:,2]) #ori

        #print(im_ori)
        im = im_ori #cv2.cvtColor(im_ori,cv2.COLOR_BGR2RGB)
        plt.imsave('./train-1-0-gifs-Rspace/'+str(i+1)+'.png', im)

        tosave = plt.imread('./train-1-0-gifs-Rspace/'+str(i+1)+'.png')
        #print("Adding frame to GIF file: ", idx + 1)
        writer.append_data(tosave)




In [ ]:
#create GIFs

In [ ]:
#RGB Space - equalized
test_img = copy.deepcopy(img[0])
testlocalbg = copy.deepcopy(local_mean_bg)
testglobalbg = copy.deepcopy(global_mean_bg)

plt.figure(figsize=(42,42))
p_background = test_img #cv2.cvtColor(np.float32(background), cv2.COLOR_BGR2LAB) #PIL is RGB, cv2 is BGR
#p_background = exposure.equalize_hist(p_background,mask=mask)
plt.subplot(3,4,1)
plt.imshow(p_background) #R
plt.subplot(3,4,2)
plt.imshow(p_background[:,:,0]) #B
plt.subplot(3,4,3)
plt.imshow(p_background[:,:,1]) #G
plt.subplot(3,4,4)
plt.imshow(p_background[:,:,2]) #R

#p_background = global_mean_bg #cv2.cvtColor(np.float32(background), cv2.COLOR_BGR2LAB) #PIL is RGB, cv2 is BGR
p_background = cv2.absdiff( np.uint8(test_img*255), np.uint8(testlocalbg*255) )
#p_background = exposure.equalize_hist(p_background,mask=mask)
plt.subplot(3,4,5)
plt.imshow(p_background) #R
plt.subplot(3,4,6)
plt.imshow(p_background[:,:,0]) #B
plt.subplot(3,4,7)
plt.imshow(p_background[:,:,1]) #G
plt.subplot(3,4,8)
plt.imshow(p_background[:,:,2]) #R

p_background = cv2.absdiff( np.uint8(test_img*255), np.uint8(testglobalbg*255) ) #cv2.cvtColor(np.float32(background), cv2.COLOR_BGR2LAB) #PIL is RGB, cv2 is BGR
#p_background = exposure.equalize_hist(p_background,mask=mask)
plt.subplot(3,4,9)
plt.imshow(p_background) #R
plt.subplot(3,4,10)
plt.imshow(p_background[:,:,0]) #B
plt.subplot(3,4,11)
plt.imshow(p_background[:,:,1]) #G
plt.subplot(3,4,12)
plt.imshow(p_background[:,:,2]) #R

In [ ]:
#LAB Space
test_img = copy.deepcopy(img[0])
testlocalbg = copy.deepcopy(local_mean_bg)
testglobalbg = copy.deepcopy(global_mean_bg)

plt.figure(figsize=(36,36))
p_background = cv2.cvtColor(np.float32(test_img), cv2.COLOR_BGR2LAB)
plt.subplot(3,3,1)
plt.imshow(p_background[:,:,0]) #B
plt.subplot(3,3,2)
plt.imshow(p_background[:,:,1]) #G
plt.subplot(3,3,3)
plt.imshow(p_background[:,:,2]) #R

p_background = cv2.absdiff( np.uint8(test_img*255), np.uint8(testlocalbg*255) )
p_background = cv2.cvtColor(np.float32(p_background), cv2.COLOR_BGR2LAB)
#p_background = cv2.cvtColor(np.float32(global_mean_bg), cv2.COLOR_BGR2LAB)
plt.subplot(3,3,4)
plt.imshow(p_background[:,:,0]) #B
plt.subplot(3,3,5)
plt.imshow(p_background[:,:,1]) #G
plt.subplot(3,3,6)
plt.imshow(p_background[:,:,2]) #R

p_background = cv2.absdiff( np.uint8(test_img*255), np.uint8(testglobalbg*255) ) #cv2.cvtColor(np.float32(background), cv2.COLOR_BGR2LAB) #PIL is RGB, cv2 is BGR
p_background = cv2.cvtColor(np.float32(p_background), cv2.COLOR_BGR2LAB)
plt.subplot(3,3,7)
plt.imshow(p_background[:,:,0]) #B
plt.subplot(3,3,8)
plt.imshow(p_background[:,:,1]) #G
plt.subplot(3,3,9)
plt.imshow(p_background[:,:,2]) #R

In [ ]:
#YCrCb Space
test_img = copy.deepcopy(img[0])
testlocalbg = copy.deepcopy(local_mean_bg)
testglobalbg = copy.deepcopy(global_mean_bg)

plt.figure(figsize=(36,36))
p_background = cv2.cvtColor(np.float32(test_img), cv2.COLOR_BGR2YCrCb)
plt.subplot(3,3,1)
plt.imshow(p_background[:,:,0]) #B
plt.subplot(3,3,2)
plt.imshow(p_background[:,:,1]) #G
plt.subplot(3,3,3)
plt.imshow(p_background[:,:,2]) #R

p_background = cv2.absdiff( np.uint8(test_img*255), np.uint8(testlocalbg*255) )
p_background = cv2.cvtColor(np.float32(p_background), cv2.COLOR_BGR2YCrCb)
#p_background = cv2.cvtColor(np.float32(global_mean_bg), cv2.COLOR_BGR2YCrCb)
plt.subplot(3,3,4)
plt.imshow(p_background[:,:,0]) #B
plt.subplot(3,3,5)
plt.imshow(p_background[:,:,1]) #G
plt.subplot(3,3,6)
plt.imshow(p_background[:,:,2]) #R

p_background = cv2.absdiff( np.uint8(test_img*255), np.uint8(testglobalbg*255) ) #cv2.cvtColor(np.float32(background), cv2.COLOR_BGR2LAB) #PIL is RGB, cv2 is BGR
p_background = cv2.cvtColor(np.float32(p_background), cv2.COLOR_BGR2YCrCb)
plt.subplot(3,3,7)
plt.imshow(p_background[:,:,0]) #B
plt.subplot(3,3,8)
plt.imshow(p_background[:,:,1]) #G
plt.subplot(3,3,9)
plt.imshow(p_background[:,:,2]) #R

In [ ]:
#HSV Space
test_img = copy.deepcopy(img[0])
testlocalbg = copy.deepcopy(local_mean_bg)
testglobalbg = copy.deepcopy(global_mean_bg)


plt.figure(figsize=(36,36))
p_background = cv2.cvtColor(np.float32(test_img), cv2.COLOR_BGR2HSV)
plt.subplot(3,3,1)
plt.imshow(p_background[:,:,0]) #B
plt.subplot(3,3,2)
plt.imshow(p_background[:,:,1]) #G
plt.subplot(3,3,3)
plt.imshow(p_background[:,:,2]) #R

p_background = cv2.absdiff( np.uint8(test_img*255), np.uint8(testlocalbg*255) )
p_background = cv2.cvtColor(np.float32(p_background), cv2.COLOR_BGR2HSV)
#p_background = cv2.cvtColor(np.float32(global_mean_bg), cv2.COLOR_BGR2HSV)
plt.subplot(3,3,4)
plt.imshow(p_background[:,:,0]) #B
plt.subplot(3,3,5)
plt.imshow(p_background[:,:,1]) #G
plt.subplot(3,3,6)
plt.imshow(p_background[:,:,2]) #R

p_background = cv2.absdiff( np.uint8(test_img*255), np.uint8(testglobalbg*255) ) #cv2.cvtColor(np.float32(background), cv2.COLOR_BGR2LAB) #PIL is RGB, cv2 is BGR
p_background = cv2.cvtColor(np.float32(p_background), cv2.COLOR_BGR2HSV)
plt.subplot(3,3,7)
plt.imshow(p_background[:,:,0]) #B
plt.subplot(3,3,8)
plt.imshow(p_background[:,:,1]) #G
plt.subplot(3,3,9)
plt.imshow(p_background[:,:,2]) #R

In [ ]:
import copy

frame_diffs = []


img_ori = copy.deepcopy(img) #make a copy of the list, but the element reference remains the same

for i in img:
    img_g = cv2.cvtColor(np.float32(i), cv2.COLOR_BGR2GRAY) 

    #apply mask
    img_g = cv2.bitwise_and(img_g,img_g,mask=mask)


    # find the difference between current frame and base frame
    frame_diff = cv2.absdiff( np.uint8(img_g*255), np.uint8(global_grey_background*255) )
    # thresholding to convert the frame to binary
    ret, thres = cv2.threshold( frame_diff , 50, 255, cv2.THRESH_BINARY)

    # dilate the frame a bit to get some more white area...
    # ... makes the detection of contours a bit easier
    dilate_frame = cv2.dilate(thres, None, iterations=2)

    # append the final result into the `frame_diff_list`
    frame_diffs += [dilate_frame]

## add all the frames in the `frame_diff_list`
sum_frames = sum(frame_diffs)

## find the contours around the white segmented areas
#contours, hierarchy = cv2.findContours(sum_frames, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

## draw the contours, not strictly necessary
#for i, contour in enumerate(contours):
#    frame = img[-1].copy()
#    cv2.drawContours(frame, contours, i, (0, 0, 255), 3) #on last frame




    
    



In [ ]:
L = len(frame_diffs)
mean_frame_diff = np.mean(np.array(frame_diffs), axis=0)
#plt.imshow(mean_frame_diff)
#img_ori_ = img_ori.copy()

img = copy.deepcopy(img_ori)

plt.figure(figsize=(50,50))
for l in range(L):
    plt.subplot(L,1,l+1)
    temp_diff = cv2.absdiff(frame_diffs[l],np.uint8(mean_frame_diff))

    ret, thres = cv2.threshold( frame_diff , 50, 255, cv2.THRESH_BINARY)
    dilate_frame = cv2.dilate(thres, None, iterations=2)
    #dilate_frame = thres

    # find the contours around the white segmented areas
    contours, hierarchy = cv2.findContours(dilate_frame, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        # continue through the loop if contour area is less than 500...
        # ... helps in removing noise detection
        if cv2.contourArea(contour) < 70:
            continue
        #print('here')
        # get the xmin, ymin, width, and height coordinates from the contours
        (x, y, w, h) = cv2.boundingRect(contour)
        # draw the bounding boxes
        cv2.rectangle(img[l], (x, y), (x+w, y+h), (0, 255, 0), 2)
    plt.imshow(img[l])

    
            

In [ ]:

img = copy.deepcopy(img_ori)
for im in img:
    plt.figure(figsize=(12,8))
    plt.imshow(cv2.cvtColor(np.float32(im), cv2.COLOR_RGB2GRAY) )

### Image Pyramids

In [ ]:
from copy import deepcopy

img = deepcopy(img_ori)

img_A = img[3]

# generate Gaussian pyramid for A
G = img_A.copy()
gpA = [G]
for i in range(6):
    G = cv2.pyrDown(G)
    gpA.append(G)

# generate Laplacian Pyramid for A
lpA = [gpA[5]]
for i in range(5,0,-1):
    GE = cv2.pyrUp(gpA[i])
    L = cv2.subtract(gpA[i-1],GE)
    lpA.append(L)

In [ ]:
lpA[0].max()

In [ ]:
img = deepcopy(img_ori)
im_invs =[]
for im in img:
    im_inv = cv2.bitwise_not(np.uint8(im*255))
    im_invs += [im_inv]
    plt.figure()
    plt.imshow(im_inv)
im_invs_avg = np.mean(im_invs, axis=0)
plt.figure()
plt.imshow( im_invs_avg /255)

In [ ]:
a = cv2.absdiff( np.uint8(im_invs_avg), np.uint8(im_invs[0]) )
plt.figure()
plt.imshow(a*5)

In [ ]:
img_g = cv2.cvtColor(np.float32(img[0]), cv2.COLOR_BGR2GRAY) 
plt.figure()
plt.imshow(img_g)

In [ ]:
im_invs_avg.max()